<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#確率計算" data-toc-modified-id="確率計算-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>確率計算</a></span><ul class="toc-item"><li><span><a href="#予備検討" data-toc-modified-id="予備検討-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>予備検討</a></span></li></ul></li></ul></div>

# 確率計算

## 予備検討

In [21]:
import numpy as np
from scipy.special import comb
from scipy.special import factorial
import pickle

In [22]:
def get_ndup( lcomb ):
    a = np.bincount( lcomb )
    a= factorial(a, exact=True )
    return np.prod( a )

In [23]:
def get_ncomb( ndepth, nopen, lcomb ):
    if  len(lcomb) == 1 :
        return ndepth
    elif ndepth == 1:
        return 0    # depthが足りないことはあり得ない
    else:
        a = get_ncomb( ndepth-1, nopen-lcomb[0], lcomb[1:])
#        print( a )
        return ndepth * comb(nopen, lcomb[0], exact=True ) *  a

In [24]:
def rmdup_ncomb( ncomb, lcomb ):
    return ncomb // get_ndup( lcomb )

In [25]:
def get_lcomb( num, ndepth ):
    ret = [[num]]
    for i in range(1,num//2+1):
        ret = ret +[ [i] + lst for lst in get_lcomb(num-i, ndepth) ]
#        ret = [ lst for lst in ret if len(lst) <= ndepth ] # depthより長いリストはあり得ない
    return ret

In [26]:
def rmdup_lcomb( lcomb, ndepth ):
    ret = [ sorted(lst) for lst in lcomb ] # 要素の中身をsort
    ret = [ tuple(lst) for lst in ret ] # sortするならtupleにする
    ret = sorted(ret ) # sort
    ret = set(ret) # uniq
    ret = [ lst for lst in ret if len(lst) <= ndepth ] # depthより長いリストはあり得ない
    return ret

In [27]:
def get_all_comb( ndepth, nopen ):
    lcomb = get_lcomb(nopen, ndepth)
    lcomb = rmdup_lcomb(lcomb, ndepth)
    res = {}
    for tpl in lcomb:
        ncomb = get_ncomb( ndepth, nopen, tpl )
        res[tpl] = rmdup_ncomb( ncomb, tpl ) 
    return res   

In [28]:
ndepth = 13
nopen = 4
res = get_all_comb( ndepth, nopen )
print( res )
print( "total = ", ndepth**nopen )
print( "sum   = ", np.sum(list(res.values())))

{(1, 3): 624, (1, 1, 1, 1): 17160, (4,): 13, (1, 1, 2): 10296, (2, 2): 468}
total =  28561
sum   =  28561


In [29]:
ndepth = 13
nopens = np.arange( 1,20)
for nopen in nopens:
    lcomb = get_lcomb(nopen, ndepth)
    lcomb = rmdup_lcomb(lcomb, ndepth)
    with open( "lcomb_%d.pkl" % (nopen), 'wb') as f:
        pickle.dump( lcomb, f)
    with open( "lcomb_%d.pkl" % (nopen), 'rb') as fo:
        lcomb2 = pickle.load( fo )
    #print( nopen )
    assert lcomb == lcomb2, "Not Matched"